### Format for web scraping prem match data

In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [63]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

In [64]:
links = standings_table.find_all('a')  # all links in the web page
links = [l.get("href") for l in links]  # get the reference to all the links
links = [l for l in links if '/squads/' in l]  # all links with '/squads/' in order to find the team pages for all clubs
team_urls = [f"https://fbref.com{l}" for l in links]  # absolute links for club specific pages

team_url = team_urls[0] # first club page link
data = requests.get(team_url)  # get data from club page

In [65]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0] # Scores and fixtures dataframe for the club
matches = matches[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Attendance', 'Captain', 'Formation','Referee']]

In [66]:
data = requests.get(team_url)
soup = BeautifulSoup(data.text) # extract all information from page
links = soup.find_all('a') # find all links on page for the club
links = [l.get("href") for l in links] # get reference urls for those links
links = [l for l in links if l and (('all_comps/shooting/' in l) or ('all_comps/possession/' in l) or ('all_comps/passing/' in l) or ('all_comps/defense/' in l))] # filter to only shooting stats links
links = [*set(links)]


['/en/squads/18bb7c10/2022-2023/matchlogs/all_comps/defense/Arsenal-Match-Logs-All-Competitions', '/en/squads/18bb7c10/2022-2023/matchlogs/all_comps/passing/Arsenal-Match-Logs-All-Competitions', '/en/squads/18bb7c10/2022-2023/matchlogs/all_comps/shooting/Arsenal-Match-Logs-All-Competitions', '/en/squads/18bb7c10/2022-2023/matchlogs/all_comps/possession/Arsenal-Match-Logs-All-Competitions']


In [67]:
df_info = {
    'matches': {
        'name': 'Matches',
        'column_levels': [],
        'df': matches,
    },
    'shooting': {
        'name': 'Shooting',
        'column_levels': ['Expected', '', 'Standard', ''],
        'df': pd.DataFrame()
    },
    'possession': {
        'name': 'Possession',
        'column_levels': ['Carries', 'Dribbles', '', 'Receiving', 'Touches', ''],
        'df': pd.DataFrame()
    },
    'passing': {
        'name': 'Passing',
        'column_levels': ['', 'Long', 'Medium', 'Short', 'Total', '', '', '', '', '', '', '', ''],
        'df': pd.DataFrame()
    },
    'defense': {
        'name': 'Defensive Actions',
        'column_levels': ['Blocks', '', 'Pressures', 'Tackles', '', '', '', '', '', 'Vs Dribbles'],
        'df': pd.DataFrame()
    }
}


In [68]:
# Update dataframes in our dictionary to pull web page tables
for l in links:
    for key, webster in df_info.items():

        if (key != 'matches') and ('all_comps/'+ key in l):
                data = requests.get(f"https://fbref.com{l}") # pull that data for the stats link
                webster['df'] = pd.read_html(data.text, match=webster['name'])[0] # read the link's data from the table
                d = dict(zip(webster['df'].columns.levels[0], webster['column_levels'])) # Clear out unecessary multi indexed column names and rename columns to specify features
                webster['df'] = webster['df'].rename(columns=d, level=0)
            

In [69]:
# Clean dataframe structure
for key, webster in df_info.items():
    if (key != 'matches'):
        webster['df'].columns = ['_'.join(col).strip() for col in webster['df'].columns.values]
    webster['df'].columns = webster['df'].columns.str.lstrip("_")
    webster['df'].columns = webster['df'].columns.str.replace(' ', '_')
    webster['df'].columns = webster['df'].columns.str.replace('-', '_')
    webster['df'].columns = webster['df'].columns.str.replace(':', '_')
    webster['df'].columns = webster['df'].columns.str.lower()
    webster['df'] = webster['df'].set_index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga', 'opponent'])
    webster['df'] = webster['df'].add_prefix(key + '_')
    webster['df'] = webster['df'].reset_index()

In [70]:
team_data = df_info['matches']['df'].copy()
for key, webseter in df_info.items():
    if (key != 'matches'):
        team_data = team_data.merge(df_info[key]['df'], on=['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga', 'opponent'])


In [77]:
# pd.set_option('max_columns', None) # set to see all columns in df
# pd.reset_option('max_columns')
team_data.head(3)


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,defense_pressures_att_3rd,defense_blocks_blocks,defense_blocks_sh,defense_blocks_shsv,defense_blocks_pass,defense_int,defense_tkl+int,defense_clr,defense_err,defense_match_report
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,42,21,6,0,15,17,NaN,20,0,Match Report
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,42,15,1,1,14,14,NaN,13,0,Match Report
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,31,12,0,0,12,17,NaN,18,1,Match Report


In [76]:
# pd.set_option('max_rows', None) # set this to seee all columns
# pd.reset_option('max_rows')
pd.DataFrame(team_data.columns)



,0
0,date
1,time
2,comp
3,round
4,day
...,...
97,defense_int
98,defense_tkl+int
99,defense_clr
100,defense_err


In [78]:
team_data[(team_data['comp']=='Premier League')]

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,defense_pressures_att_3rd,defense_blocks_blocks,defense_blocks_sh,defense_blocks_shsv,defense_blocks_pass,defense_int,defense_tkl+int,defense_clr,defense_err,defense_match_report
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,42,21,6,0,15,17,NaN,20,0,Match Report
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,42,15,1,1,14,14,NaN,13,0,Match Report
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,31,12,0,0,12,17,NaN,18,1,Match Report
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,25,12,2,0,10,15,NaN,8,1,Match Report
